<a href="https://colab.research.google.com/github/Neoneto/Diamond_Analysis/blob/main/Data_PreProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Pre Processing for the Diamonds Dataset
Dataset for this project is separated into different CSV files. In this notebook, the files are combined in a single CSV file. It is then cleaned. Finally, saved as a new CSV file.

# Import Libraries


In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import glob
import os
import re

# Import Data


In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define the path to the CSV files
path = '/content/drive/My Drive/Coding Dojo/Diamonds/CSVs'                
all_files = glob.glob(os.path.join(path, "*.csv"))     

# Read all the CSV files in the folder
df_from_each_file = (pd.read_csv(f) for f in all_files)
df0  = pd.concat(df_from_each_file, ignore_index=True)

# Show the first few rows of combined dataframe
df0.head()

,Id,Shape,Weight,Clarity,Colour,Cut,Polish,Symmetry,Fluorescence,Messurements,Price
0,1638147,CUSHION,0.55,SI2,E,EX,EX,VG,N,5.05-4.35×2.94,"1,378.65"
1,1630155,CUSHION,0.50,VVS1,FANCY,EX,EX,VG,F,4.60-4.31×2.92,"1,379.74"
2,1612606,CUSHION,0.51,VS2,H,EX,EX,VG,N,4.71-4.35×2.94,"1,380.19"
3,1638140,CUSHION,0.50,VS2,H,EX,EX,VG,N,4.91-4.26×2.88,"1,380.61"
4,1536093,CUSHION,0.53,SI1,D,EX,VG,VG,N,4.70-4.46×3.01,"1,383.13"


In [ ]:
# Check the info of the dataframe
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6339 entries, 0 to 6338
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Id            6339 non-null   object 
 1   Shape         6339 non-null   object 
 2   Weight        6339 non-null   float64
 3   Clarity       6319 non-null   object 
 4   Colour        6339 non-null   object 
 5   Cut           6337 non-null   object 
 6   Polish        6338 non-null   object 
 7   Symmetry      6332 non-null   object 
 8   Fluorescence  6337 non-null   object 
 9   Messurements  6339 non-null   object 
 10  Price         6339 non-null   object 
dtypes: float64(1), object(10)
memory usage: 544.9+ KB


There are few missing values.

In [ ]:
# Count the missing values
df0.isnull().sum()

Id               0
Shape            0
Weight           0
Clarity         20
Colour           0
Cut              2
Polish           1
Symmetry         7
Fluorescence     2
Messurements     0
Price            0
dtype: int64

# Data Wrangling

Since we have more than 6000 rows and just a few rows with missing values, we can just drop those rows with missing values. Developing a method to fill those missing values may not give much improvement in the data and this may even affect the results.

In [ ]:
# Drop rows with missing values
df0.dropna(inplace = True)
df0.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6311 entries, 0 to 6338
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Id            6311 non-null   object 
 1   Shape         6311 non-null   object 
 2   Weight        6311 non-null   float64
 3   Clarity       6311 non-null   object 
 4   Colour        6311 non-null   object 
 5   Cut           6311 non-null   object 
 6   Polish        6311 non-null   object 
 7   Symmetry      6311 non-null   object 
 8   Fluorescence  6311 non-null   object 
 9   Messurements  6311 non-null   object 
 10  Price         6311 non-null   object 
dtypes: float64(1), object(10)
memory usage: 591.7+ KB


In [ ]:
# rename some columns
df0.rename(columns = {'Messurements': 'Measurements',
                      'Colour': 'color_grade'}, inplace = True)
df0.head(1)

,Id,Shape,Weight,Clarity,color_grade,Cut,Polish,Symmetry,Fluorescence,Measurements,Price
0,1638147,CUSHION,0.55,SI2,E,EX,EX,VG,N,5.05-4.35×2.94,"1,378.65"


Create new columns, average width (arithmetic mean of the widest and narrowest face) and the depth.




In [ ]:
# Unpack the measurements column into the physical dimensions of the diamond into 3 separate columns
df0['max_width'] = df0.Measurements.apply(lambda x: np.array(re.findall('\d+\.\d+', x)).astype(float)[:2].max()) #Take the larger width as the max width
df0['min_width'] = df0.Measurements.apply(lambda x: np.array(re.findall('\d+\.\d+', x)).astype(float)[:2].min())
df0['depth'] = df0.Measurements.str.findall(r'\d+\.\d+',).str[2].astype(float) # last measurement as depth

# drop the Measurements column
df0.drop(columns = 'Measurements', inplace = True) # drop the original column

# Check the result
df0.head()


,Id,Shape,Weight,Clarity,color_grade,Cut,Polish,Symmetry,Fluorescence,Price,max_width,min_width,depth
0,1638147,CUSHION,0.55,SI2,E,EX,EX,VG,N,"1,378.65",5.05,4.35,2.94
1,1630155,CUSHION,0.50,VVS1,FANCY,EX,EX,VG,F,"1,379.74",4.60,4.31,2.92
2,1612606,CUSHION,0.51,VS2,H,EX,EX,VG,N,"1,380.19",4.71,4.35,2.94
3,1638140,CUSHION,0.50,VS2,H,EX,EX,VG,N,"1,380.61",4.91,4.26,2.88
4,1536093,CUSHION,0.53,SI1,D,EX,VG,VG,N,"1,383.13",4.70,4.46,3.01


# Inconsistencies and data type

In [ ]:
# print all unique elements for each column
for c_name in df0.columns:
  if df0[c_name].dtypes == 'O':
    print(c_name)
    print(df0[c_name].unique())

Id
['1638147' '1630155' '1612606' ... '1789435' '1774887' '1630512']
Shape
['CUSHION' 'HEART' 'MARQUISE' 'OVAL' 'PEAR' 'PRINCESS' 'EMERALD' 'ROUND']
Clarity
['SI2' 'VVS1' 'VS2' 'SI1' 'VVS2' 'VS1' 'I3' 'I1' 'IF' 'I2' 'FL']
color_grade
['E' 'FANCY' 'H' 'D' 'I' 'M' 'K' 'W-X' 'Y-Z' 'N' 'J' 'F' 'L' 'G' 'O-P'
 'S-T' 'U-V' 'W' 'Q-R' 'O']
Cut
['EX' 'VG' 'GD' 'F']
Polish
['EX' 'VG' 'GD' 'F']
Symmetry
['VG' 'EX' 'GD' 'FR']
Fluorescence
['N' 'F' 'ST' 'M' 'VS' 'SL' 'VSL']
Price
['1,378.65' '1,379.74' '1,380.19' ... '2,453.20' '2,453.41' '2,453.69']


By manual inspection, we can see that there are possible inconsistencies in the categorical columns. However, the Price column is encoded as a string.

In [ ]:
# change price to float
from decimal import Decimal
import babel.numbers

# Price is formatted as a string, here it is read as a money format and converted to float
df0['Price'] = df0['Price'].replace('[\$,]', '', regex=True).astype(float)

# Check the resulting dataframe
df0.head()

,Id,Shape,Weight,Clarity,color_grade,Cut,Polish,Symmetry,Fluorescence,Price,max_width,min_width,depth
0,1638147,CUSHION,0.55,SI2,E,EX,EX,VG,N,1378.65,5.05,4.35,2.94
1,1630155,CUSHION,0.50,VVS1,FANCY,EX,EX,VG,F,1379.74,4.60,4.31,2.92
2,1612606,CUSHION,0.51,VS2,H,EX,EX,VG,N,1380.19,4.71,4.35,2.94
3,1638140,CUSHION,0.50,VS2,H,EX,EX,VG,N,1380.61,4.91,4.26,2.88
4,1536093,CUSHION,0.53,SI1,D,EX,VG,VG,N,1383.13,4.70,4.46,3.01


In [ ]:
# Create a new column price_per_carat
# Quotient of the Price and the weight of the diamond
df0['price_per_carat'] = df0.Price/df0.Weight

# Check the final dataframe
df0.head()

,Id,Shape,Weight,Clarity,color_grade,Cut,Polish,Symmetry,Fluorescence,Price,max_width,min_width,depth,price_per_carat
0,1638147,CUSHION,0.55,SI2,E,EX,EX,VG,N,1378.65,5.05,4.35,2.94,2506.636364
1,1630155,CUSHION,0.50,VVS1,FANCY,EX,EX,VG,F,1379.74,4.60,4.31,2.92,2759.480000
2,1612606,CUSHION,0.51,VS2,H,EX,EX,VG,N,1380.19,4.71,4.35,2.94,2706.254902
3,1638140,CUSHION,0.50,VS2,H,EX,EX,VG,N,1380.61,4.91,4.26,2.88,2761.220000
4,1536093,CUSHION,0.53,SI1,D,EX,VG,VG,N,1383.13,4.70,4.46,3.01,2609.679245


In [ ]:
# Save the preprocessed dataframe as a new csv file.
df0.to_csv("/content/drive/My Drive/Coding Dojo/Diamonds/diamonds_preprocessed.csv", index = False)